In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile
import os

zip_file_path = '/content/drive/MyDrive/chest_xray/chest_xray.zip'

extract_path = '/content/drive/MyDrive'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)


In [4]:
!cd /content/drive/MyDrive/chest_xray

In [3]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.preprocessing import binarize


In [5]:
# Gabor filtresi parametreleri
ksize = 10 # Çekirdek boyutu
sigma = 1.35 # Gauss fonksiyonunun standart sapması
theta = np.pi   # Yönelim (0, pi)
lambd = np.pi / 4 # Dalga boyu (frekans)
gamma = 5.0  # Uzaysal en-boy oranı
psi = 0  # Faz ofseti
boyut = 64
# Gabor çekirdeğini oluştur
gabor_kernel = cv2.getGaborKernel((ksize, ksize), sigma, theta, lambd, gamma, psi,  ktype=cv2.CV_32F)

X_train = []
y_train = []

X_test = []
y_test = []

In [6]:
def X_y(klasor_adi,vector, disease):
  if (klasor_adi == 'train'):
    X_train.append(vector)
    y_train.append(disease)
  else:
    X_test.append(vector)
    y_test.append(disease)

klasor_names = ['train', 'test']
train_say=0

for klasor_name in klasor_names:
  klsr_path = os.path.join('/content/drive/MyDrive/chest_xray', klasor_name)
  for foldername in os.listdir(klsr_path):
    diseasepath = os.path.join(klsr_path, foldername)
    for filename in os.listdir(diseasepath):
      img_path = os.path.join(diseasepath, filename)
      image = cv2.imread(img_path)

      image = cv2.resize(image, (boyut,boyut))
      image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

      gabor_filtered_image = cv2.filter2D(image, cv2.CV_8UC3, gabor_kernel)
      gabor_flat = gabor_filtered_image.flatten()

      edges = cv2.Canny(image, 50, 50)
      edge_flat = edges.flatten()

      merged_vec = gabor_flat + edge_flat
      if klasor_name == 'train':
        train_say += 1
      X_y(klasor_name, merged_vec, foldername)




In [7]:
def predict_Knn(k, size, test_arr):

  y_knn_predict = []
  for test in test_arr:
    pre = []
    d_dict = {'covid': 0, 'normal': 0, 'pneumonia': 0}
    for i in range(train_say):
      top = 0
      for j in range(len(test)):
        top += abs(test[j] - X_train[i][j])
      pre.append((top,y_train[i]))
    pre.sort()
    for idx in range(k):
      d_dict[pre[idx][1]] += 1

    max_pos_dis = 'covid'
    for dis in d_dict:
      if d_dict[dis] > d_dict[max_pos_dis]:
        max_pos_dis = dis

    y_knn_predict.append(max_pos_dis)

  y_k_pred =  np.array(y_knn_predict)




In [8]:
X_train = np.array(X_train)
y_train = np.array(y_train)

X_test = np.array(X_test)
y_test = np.array(y_test)


In [9]:
print(len(X_train[0]))
print(len(X_test[0]))
print(train_say)

4096
4096
3000


In [10]:
import tensorflow as tf

In [16]:

knn = KNeighborsClassifier(n_neighbors= 3, p=1)
knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)
print(y_pred)

accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

precision = precision_score(y_test, y_pred, average ='weighted')
print(precision)

recall = recall_score(y_test, y_pred, average = 'macro')
print(recall)


"""
y_k_pred = []
with tf.device('/gpu:0'):
  predict_Knn(k=3, size=boyut, test_arr = X_test)

print(y_k_pred)

accuracy = accuracy_score(y_test, y_k_pred)
print(accuracy)
"""

['pneumonia' 'pneumonia' 'pneumonia' 'pneumonia' 'pneumonia' 'pneumonia'
 'pneumonia' 'covid' 'pneumonia' 'pneumonia' 'pneumonia' 'pneumonia'
 'pneumonia' 'covid' 'pneumonia' 'pneumonia' 'covid' 'pneumonia'
 'pneumonia' 'pneumonia' 'pneumonia' 'pneumonia' 'pneumonia' 'pneumonia'
 'covid' 'pneumonia' 'pneumonia' 'covid' 'covid' 'covid' 'pneumonia'
 'pneumonia' 'pneumonia' 'pneumonia' 'pneumonia' 'pneumonia' 'covid'
 'covid' 'covid' 'pneumonia' 'pneumonia' 'covid' 'pneumonia' 'covid'
 'pneumonia' 'pneumonia' 'covid' 'pneumonia' 'pneumonia' 'covid'
 'pneumonia' 'covid' 'pneumonia' 'pneumonia' 'pneumonia' 'covid'
 'pneumonia' 'pneumonia' 'pneumonia' 'pneumonia' 'pneumonia' 'pneumonia'
 'pneumonia' 'pneumonia' 'pneumonia' 'pneumonia' 'pneumonia' 'pneumonia'
 'pneumonia' 'pneumonia' 'pneumonia' 'pneumonia' 'pneumonia' 'pneumonia'
 'pneumonia' 'pneumonia' 'pneumonia' 'pneumonia' 'pneumonia' 'pneumonia'
 'covid' 'pneumonia' 'pneumonia' 'pneumonia' 'normal' 'pneumonia' 'covid'
 'pneumonia' 'pne

"\ny_k_pred = []\nwith tf.device('/gpu:0'):\n  predict_Knn(k=3, size=boyut, test_arr = X_test)\n\nprint(y_k_pred)\n\naccuracy = accuracy_score(y_test, y_k_pred)\nprint(accuracy)\n"